<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Assignment 2*

# Document Representations: Bag-Of-Words

In [52]:
import re
import string

!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# For Web Scraping
!pip install lxml
!pip install bs4
import requests
import bs4
from bs4 import BeautifulSoup

import time

Requirement already up-to-date: nltk in /anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages (3.4.4)


[nltk_data] Downloading package punkt to /Users/azel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/azel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached https://files.pythonhosted.org/packages/1d/16/c220beb03c4b4609924f0fc1bee5e0e5e8082089103fdc0b2a7c8b8bb003/lxml-4.3.4-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl


# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

If you choose to not to scrape the data, there is a CSV with outdated data in the directory. Remeber, if you scrape Indeed, you're helping yourself find a job. ;)

In [38]:
##### Your Code Here #####
URL = 'https://www.indeed.com/jobs?q=data+scientist+$20,000&l=Virginia&start=0'

# Conduct a request of the stated URL above: 

page = requests.get(URL)

# Specify a desired format of "page" using the html parset
# This allows python to read the various compenents of the page
# Instead of treating it as on long string

soup = BeautifulSoup(page.text, "html.parser")

#printing soup in a more structured tree format that makes for easier reading

print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="/s/e7e2a1c/en_US.js" type="text/javascript">
  </script>
  <link href="/s/684a333/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&amp;l=Virginia" rel="alternate" title="Data Scientist $20,000 Jobs, Employment in Virginia" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+scientist+%2420%2C000&amp;l=Virginia" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/m/jobs?q=data+scientist+%2420%2C000&amp;l=Virginia" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] = [];
    }

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'

Target Information:

- Job Title
- Company Name
- Location
- Salary
- Job Summary

In [39]:
def extract_job_title(soup=soup):
    """
    As noted above, I could tell that the entirety of each job 
    posting is nested under <div> tags, with an attribute 
    “class” = “row result”.

    From there, I could see that job titles are listed under <a> tags, 
    with attribute “title = (title)”. One can find the value of a tag’s 
    attribute with tag[“attribute”], so I could use this to find the 
    job title for each posting.

    My function for withdrawing job title information involved three steps:

        - pulling out all <div> tags with class including “row”
        - identifying <a> tags with attribute “data-tn-element”:”jobTitle”
        - for each of these <a> tags, find the value of attributes “title”
    """
    
    jobs = []
    
    # iterate through the job listings. 
    for div in soup.find_all(name='div', 
                             attrs={"class":"row"}):
        
        # Pull the job title data
        for a in div.find_all(name="a",attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a['title'])
            
    return jobs

In [40]:
def extract_company(soup=soup):
    """
    Company names were a bit tricky, as most would appear in 
    <span> tags, with “class”:”company”. Rarely, however, they 
    will be housed in <span> tags with “class”:”result-link-source”.

    if/else statements extract the company info from either of these 
    places. Company names are output with a lot of white space around 
    them, so inputting .strip() at the end helps to remove this when 
    extracting the information.
    """
    
    
    companies = []
    
    # Iterate through the job listings
    for div in soup.find_all(name='div', 
                             attrs={'class':'row'}):
        company = div.find_all(name='span',
                               attrs={'class':'company'})
        
        # for companies housed in the company class
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
            
            # in case the company name is housed in the "span" class
            else:
                sec_try = div.find_all(name='span',
                                       attrs={'class':'result-link-source'})
                
                for span in sec_try:
                    companies.append(span.text.strip())
    
    return companies

In [44]:
def extract_location(soup=soup):
    """
    Locations are located under the <span> tags. Span tags 
    are sometimes nested within each other, such that the 
    location text may sometimes be within “class” : “location” 
    attributes, or nested in “itemprop” : “addressLocality”. 
    
    However, a simple for loop can examine all span tags for 
    text wherever it may be and retrieve the necessary information.
    """
    
    locations = []
    
    # find all spans with location attribute
    spans = soup.find_all('span', 
                          attrs={'class', 'location'})
    
    # list comprehension to scrape
    for span in spans: 
        if len(span) == 0:
            locations.append(np.nan)
        else:
            locations.append(span.text)
    
    return locations

In [46]:
extract_location()

['Tysons, VA',
 'Herndon, VA 20170',
 'Herndon, VA',
 'Williamsburg, VA',
 'Vienna, VA 22182',
 'Rosslyn, VA',
 'Herndon, VA 20171',
 'Tysons Corner, VA',
 'Alexandria, VA 22314 (Southwest Quadrant area)',
 'Richmond, VA']

In [48]:
max_results_per_location = 10

locations = ["Washington%2C+DC", "Virginia"]
columns = ["location_field", "job_title", "company_name",
           "location", "summary"]

sample_df = pd.DataFrame(columns = columns)

In [59]:
for loc in locations:
    for start in range(0, max_results_per_location, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + 
                            str(loc) + 
                            '&start=' + 
                            str(start))
        
        # Make sure I don't freak Indeed.com out
        time.sleep(1)
        
        # Setting up Beatuiful Soup
        soup = BeautifulSoup(page.text, 
                             'html.parser', 
                             from_encoding='utf-8')
        
        # Big Loop!
        for div in soup.find_all(name='div', attrs={'class':'row'}):
            # specifying the row number for the dataframe
            num = (len(sample_df) + 1)
            # Empty list for big list comprehension!
            job_post = []
            # append location name
            job_post.append(loc)
            # get job title
            for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                job_post.append(a['title'])
            # get company name
            company = div.find_all(name='span', attrs={'class':'company'})
            if len(company) > 0:
                for b in company: 
                    job_post.append(b.text.strip())
                else:
                    sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                    for span in sec_try:
                        job_post.append(span.text)
            # get location name
            try:
                c = div.findAll('span', attrs={'class': 'location'})
                for span in c:
                    job_post.append(span.text)
            except:
                job_post.append(np.nan)
            # get summary text
            try:
                d = div.findAll('span', attrs={'class': 'summary'})
                for span in d:
                    job_post.append(span.text.strip())
            except:
                job_post.append(np.nan)
                
            sample_df.loc[num] = job_post
                
            

ValueError: cannot set a row with mismatched columns

In [60]:
d

[]

In [61]:
job_post

['Washington%2C+DC', 'Data Scientist', 'WPA Opinion Research']

In [56]:
columns

['city', 'job_title', 'company_name', 'location', 'summary']

In [ ]:
 #grabbing summary text
    d = div.findAll(‘span’, attrs={‘class’: ‘summary’}) 
      for span in d:
        job_post.append(span.text.strip()) 
    #grabbing salary
    try:
      job_post.append(div.find(‘nobr’).text) 
    except:
      try:
        div_two = div.find(name=”div”, attrs={“class”:”sjcl”}) 
        div_three = div_two.find(“div”) 
        job_post.append(div_three.text.strip())
      except:
        job_post.append(“Nothing_found”) 
    #appending list of job post info to dataframe at index num
    sample_df.loc[num] = job_post



for city in city_set:
  for start in range(0, max_results_per_city, 10):
  page = requests.get(‘http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + ‘&start=’ + str(start))
  time.sleep(1)  #ensuring at least 1 second between page grabs
  soup = BeautifulSoup(page.text, “lxml”, from_encoding=”utf-8")
  for div in soup.find_all(name=”div”, attrs={“class”:”row”}): 
    #specifying row num for index of job posting in dataframe
    num = (len(sample_df) + 1) 
    #creating an empty list to hold the data for each posting
    job_post = [] 
    #append city name
    job_post.append(city) 
    #grabbing job title
    for a in div.find_all(name=”a”, attrs={“data-tn-element”:”jobTitle”}):
      job_post.append(a[“title”]) 
    #grabbing company name
    company = div.find_all(name=”span”, attrs={“class”:”company”}) 
    if len(company) > 0: 
      for b in company:
        job_post.append(b.text.strip()) 
    else: 
      sec_try = div.find_all(name=”span”, attrs={“class”:”result-link-source”})
      for span in sec_try:
        job_post.append(span.text) 
    #grabbing location name
    c = div.findAll(‘span’, attrs={‘class’: ‘location’}) 
    for span in c: 
      job_post.append(span.text) 
   

In [42]:
len(extract_job_title()), len(extract_company()), len(extract_location())

(19, 19, 10)

## 2) Use Spacy to tokenize / clean the listings 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

# 4) Visualize the most common word counts

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

 # 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.